[View in Colaboratory](https://colab.research.google.com/github/AleHRejon/tesis_data/blob/master/Tesis_030518.ipynb)

In [4]:
import numpy

X_train = numpy.load("X_train.npy")
X_test = numpy.load("X_test.npy")
y_train = numpy.load("y_train.npy")
y_test = numpy.load("y_test.npy")

FileNotFoundError: ignored

In [6]:
from google.colab import files
uploaded = files.upload(y_train.npy)

NameError: ignored

In [0]:
# TfidfVectorizer + SVC

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV

lsa_gender = TruncatedSVD(algorithm="randomized", random_state=2410)
norm_gender = Normalizer(norm = 'l2', copy = False)
clf_gender = SVC(random_state = 2410)

pipe_svc_tfidf = Pipeline([('lsa', lsa_gender), ('normal', norm_gender), ('svm', clf_gender)])


#CountVectorizer(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None, lowercase=True, 
#                preprocessor=None, tokenizer=None, stop_words=None, token_pattern=’(?u)\b\w\w+\b’,
#                ngram_range=(1, 1), analyzer=’word’, max_df=1.0, min_df=1, max_features=None, vocabulary=None,
#                binary=False, dtype=<class ‘numpy.int64’>)

#TfidfVectorizer(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None, lowercase=True,
#                preprocessor=None, tokenizer=None, analyzer=’word’, stop_words=None, token_pattern=’(?u)\b\w\w+\b’, 
#                ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, 
#                dtype=<class ‘numpy.int64’>, norm=’l2’, use_idf=True, smooth_idf=True, sublinear_tf=False)

#Normalizer(norm=’l2’, copy=True)

#SGDClassifier(loss=’hinge’, penalty=’l2’, alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=None, 
#              tol=None, shuffle=True, verbose=0, epsilon=0.1, n_jobs=1, random_state=None, learning_rate=’optimal’,
#              eta0=0.0, power_t=0.5, class_weight=None, warm_start=False, average=False, n_iter=None)

#SVC(C=1.0, kernel=’rbf’, degree=3, gamma=’auto’, coef0=0.0, shrinking=True, probability=False, tol=0.001,
#    cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape=’ovr’, random_state=None)

#LinearSVC(penalty=’l2’, loss=’squared_hinge’, dual=True, tol=0.0001, C=1.0, multi_class=’ovr’, fit_intercept=True,
#          intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=1000)

#RandomizedSearchCV(estimator, param_distributions, n_iter=10, scoring=None, fit_params=None, n_jobs=1, iid=True, 
#                   refit=True, cv=None, verbose=0, pre_dispatch=‘2*n_jobs’, random_state=None, error_score=’raise’, 
#                   return_train_score=’warn’)


param_dist = {
              "lsa__n_components": [30, 50, 100, 200, 300],
              "lsa__n_iter": [5, 10, 15],
              "svm__C": [5, 10, 100],
              "svm__kernel": ['linear', 'rbf', 'poly', 'sigmoid'],
              "svm__class_weight": ["balanced", None]}

# run randomized search
n_iter_search = 20
random_search_svc_tfidf = RandomizedSearchCV(pipe_svc_tfidf, param_distributions=param_dist, cv = 10,
                                   n_iter=n_iter_search, scoring = 'accuracy')


start = time()
random_search_svc_tfidf.fit(X_train_tfidf, y_train)
print("RandomizedSearchCV took {} minutes".format((time() - start)/60))



# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = numpy.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            
            
report(random_search_svc_tfidf.cv_results_)


from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

y_pred = random_search_svc_tfidf.predict(X_test_tfidf)
acc4 = accuracy_score(y_test, y_pred)
report4 = classification_report(y_test, y_pred)
print(acc4, report4)